# Concentration Maps
This notebook details how to combine the $T_1$-maps from the Look-Locker and Mixed sequences, and to estimate tracer concentration maps from them.
We will assume that you've already registered the $T_1$-maps from both the [Look-Locker](./looklocker-processing.ipynb) and [Mixed](./mixed-t1map-processing.ipynb) sequences, and that you have created brain segmentations using the [FreeSurfer/FastSurfer](./free-fast-surfer.ipynb) pipeline.

The steps include:
- Generating a CSF-mask and an intracranial mask.
- Combining the Mixed and Look-Locker T1-maps into a hybrid T1-map.
- Estimating intracranial concentrations.

**Expected input data**:

- `mri_processed_data/sub-01/registered/sub-01_ses-01_acq-mixed_T1map_registered.nii.gz`
- `mri_processed_data/sub-01/registered/sub-01_ses-01_acq-looklocker_T1map_registered.nii.gz`
- `mri_processed_data/sub-01/registered/sub-01_ses-01_T2w_registered.nii.gz`
- `mri_processed_data/fastsurfer/sub-01/mri/aseg.mgz`

## CSF Mask Generation

The CSF-mask is generated on the basis of the T2-weighted image, which has large contrast differences between CSF and everything else, and is also of a higher resolution than most of the other images.
Before generating the mask, we need to register it to the reference space of the pre-contrast T1-weighted image:
```bash
greedy -d 3 -a \
  -i mri_processed_data/sub-01/registered/sub-01_ses-01_T1w_registered.nii.gz \
      mri_dataset/sub-01/ses-01/anat/sub-01_ses-01_T2w.nii.gz
  -o mri_processed_data/sub-01/transforms/sub-01_ses-01_T2w.mat \
  -ia-image-centers \
  -dof 6 \
  -m NMI \
  -threads 6

greedy -d 3
  -rf mri_processed_data/sub-01/registered/sub-01_ses-01_T1w_registered.nii.gz \
  -ri NN \
  -rm mri_dataset/sub-01/ses-01/anat/sub-01_ses-01_T2w.nii.gz \
       mri_processed_data/sub-01/registered/sub-01_ses-01_T2w_registered.nii.gz \
  -r mri_processed_data/sub-01/transforms/sub-01_ses-01_T2w.mat \
  -threads 4
```

Then we generate a binary CSF-mask with the command
```bash
gmri2fem seg mask-csf \
  --input mri_processed_data/sub-01/registered/sub-01_ses-01_T2w_registered.nii.gz \
  --output mri_processed_data/sub-01/segmentations/sub-01_seg-csf_binary.nii.gz
```

## Combining T1-maps
Now we want to combine the Look-Locker and Mixed sequence $T1$-maps into hybrid T1-map, which should precisely represent T1-times in both CSF and brain tissue:
```bash
gmri2fem mri hybrid-t1map 
    --ll mri_processed_data/sub-01/registered/sub-01_ses-02_acq-looklocker_T1map_registered.nii.gz \
    --mixed mri_processed_data/sub-01/registered/sub-01_ses-02_acq-mixed_T1map_registered.nii.gz \
    --csfmask mri_processed_data/sub-01/segmentations/sub-01_seg-csf_binary.nii.gz \
    --output mri_processed_data/sub-01/T1maps/sub-01_ses-02_T1map_hybrid.nii.gz \
    --threshold 1500 \
    --erode 1
```

## Intracranial Mask Generation

Allthough the hybrid T1-maps are generated to be precise in CSF and brain parenchyma, the images still depict
surrounding tissue from the head and neck.
For the concentration estimates, we are only interested in the intracranial space which confines the visible contrast agent.

The intracranial mask is generated by combining the CSF mask generated above, with a brain mask based on the segmentations generated by FastSurfer.
First we need to resample the FastSurfer segmentation, which may not have the same resolution as the reference space.
This can be achieved by the below segmentation refinement.
This command resamples the image, uses the CSF-mask to refine the boundary between CSF and brain tissue, it segments the CSF-space, and smooths out artifacts in the interface between segments, that may be introduced during resampling. 
This is, admittedly, a bit overkill for the purpose described in this section, but at least it gets the job done.
```bash
gmri2fem seg refine \
  --fs_seg mri_processed_data/fastsurfer/sub-01/mri/wmparc.mgz \
  --reference mri_processed_data/sub-01/registered/sub-01_ses-01_T1w_registered.nii.gz \
  --csfmask mri_processed_data/sub-01/segmentations/sub-01_seg-csf_binary.nii.gz \
  --output_seg mri_processed_data/sub-01/segmentations/sub-01_seg-wmparc_refined.nii.gz \
  --output_csfseg mri_processed_data/sub-01/segmentations/sub-01_seg-csf-wmparc.nii.gz \
  --label_smoothing 1.0
```
Based on the refined segmentation from above, we can generate the intracranial mask as follows:

```bash
mri2fem seg mask-intracranial \
    --csfmask mri_processed_data/sub-01/segmentations/sub-01_seg-csf-aseg.nii.gz \
    --brain_seg mri_processed_data/sub-01/segmentations/sub-01_seg-wmparc_refined.nii.gz \
    --output mri_processed_data/sub-01/segmentations/sub-01_seg-intracranial_binary.nii.gz
```

## Concentration Maps
Finally, we can compare the hybrid T1-maps from the pre-contrast session and the later sessions, to output intracranial concentration maps by the command:
```
gmri2fem mri concentration \
    --input mri_processed_data/sub-01/T1maps/sub-01_ses-02_T1map_hybrid.nii.gz \
    --reference mri_processed_data/sub-01/T1maps/sub-01_ses-01_T1map_hybrid.nii.gz \
    --output mri_processed_data/sub-01/concentrations/sub-01_ses-02_concentration.nii.gz \
    --r1 0.0032 \
    --mask mri_processed_data/sub-01/segmentations/sub-01_seg-intracranial_binary.nii.gz
```